<a href="https://colab.research.google.com/github/AlonaMATI/Python-project-Analyze-the-results-of-A-B-testing/blob/main/Analyze_the_results_of_A_B_testing_using_statistical_methods_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("Data_AB_testing.csv")
df.head()

In [ ]:
#Визначаємо ключові метрики
METRICS = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account']
SESSION_EVENT = 'session'

#Фільтруємо лише потрібні події
df_filtered = df[df['event_name'].isin(METRICS + [SESSION_EVENT])]
df_filtered.head()

In [ ]:
#Функція для Z-тесту пропорцій
def z_test(p1, n1, p2, n2):
    if n1 == 0 or n2 == 0:
        return np.nan, np.nan
    p_pool = (p1 * n1 + p2 * n2) / (n1 + n2)
    se = np.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
    if se == 0:
        return np.nan, np.nan
    z = (p1 - p2) / se
    p_value = 2 * (1 - norm.cdf(abs(z)))
    return z, p_value

In [ ]:
# Формуємо фінальний список з результатами для кожного тесту та метрики
final_results = []
test_numbers = df_filtered['test'].unique()

# Цикл по кожному тесту
for test_num in test_numbers:
    df_test = df_filtered[df_filtered['test'] == test_num]
# Цикл по кожній метриці та групі
    for metric in METRICS:
        for group in [1, 2]:
            df_group = df_test[df_test['test_group'] == group]

            # Розрахунок "лічильника" і "знаменника"
            metric_sum = df_group[df_group['event_name'] == metric]['value'].sum()
            session_sum = df_group[df_group['event_name'] == SESSION_EVENT]['value'].sum()

            # Збереження в окремі змінні для контрольної і тестової груп
            if group == 1:
                control_metric = metric_sum
                control_session = session_sum
            else:
                test_metric = metric_sum
                test_session = session_sum

        # Розрахунок конверсії та різниці (наскільки змінилась метрика в тесті)
        cr_test = test_metric / test_session if test_session else 0
        cr_control = control_metric / control_session if control_session else 0
        metric_change = (cr_test - cr_control) / cr_control * 100


        # Z-test
        z, p = z_test(cr_test, test_session, cr_control, control_session)

        #Збереження результатів до фінальної таблиці
        final_results.append({
            'test_number': test_num,
            'metric': metric.replace(' ', '_'),
            'numerator_event': test_metric,
            'denominator_event': test_session,
            'numerator_control': control_metric,
            'denominator_control': control_session,
            'conversion_rate_test': cr_test,
            'conversion_rate_control': cr_control,
            'metric_change': metric_change,
            'z_stat': z,
            'p_value': p,
            'significant': p < 0.05
        })

# Формуємо фінальний датафрейм
final_df = pd.DataFrame(final_results)
print(final_df)

In [ ]:
#Збереження результатів в csv файл
final_df.to_csv('ab_test_results.csv', index=False)
from google.colab import files
files.download("ab_test_results.csv")

In [ ]:
final_df.to_excel('AB_test_table.xlsx')
files.download("AB_test_table.xlsx")

***Висновки по результатам кожного з тестів:***

Тест №1:
Три метрики з чотирьох мають приріст і їх зростання є статистично значущим, а отже вплив тесту позитивний і його варто запроваджувати.

Тест №2: всі метрики мають слабку позитивну динаміку, але різниця не є статистично значущою, тому тест запроваджувати не рекомендується.

Тест №3: Має негативний ефект на ключову метрику begin_checkout. Запроваджувати не рекомендується.

Тест №4:
Має негативний вплив на 2 метрики з чотирьох, зниження яких є до того ще й значуще. Запроваджувати тест не рекомендується.

Посилання на csv файл з результатами:

https://drive.google.com/file/d/1XEyMxxtZGeF04FshRci5JiJ90z7x7foH/view?usp=sharing

Посилання на дашборд:

https://public.tableau.com/views/Book1_17523343694510/DBSignificant?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link